In [1]:
import requests
from collections import defaultdict
from tqdm import tqdm
from review_filter import ReviewFilter
from preprocess import clean_text, get_no_space_length
import pandas as pd

In [2]:
filter = ReviewFilter()

def text_cleaning(text):
    text = clean_text(text, remove_dot=False)
    text = filter.filter_text(text, verbose=True, join=True)
    return text

In [3]:
products = requests.get(f"http://localhost:8000/api/product/summary/all/").json()
reviews = requests.get(f"http://localhost:8000/api/review/all/").json()

In [4]:
review_dict = defaultdict(str)
for review in tqdm(reviews['product']):
    review_dict[review['prod_id']] += filter.filter_text((review['context'] + ". "), verbose=True, join=True)

product_dict = {}
for product in products['product']:
    if product['unique_product_id'] in product_dict:
        continue
    if not product['summary']:
        continue
    
    product_dict[product['unique_product_id']] = (product['summary'][1:-1], review_dict[product['product_id']])

100%|██████████| 77732/77732 [07:40<00:00, 168.80it/s]


In [5]:
ids = []
summary = []
context = []
for u_id, item in product_dict.items():
    ids.append(u_id)
    summary.append(item[0])
    context.append(item[1])

In [6]:
train = pd.DataFrame({"id": ids, "context": context, "summary": summary})
train.to_csv('./train.csv', index=False)

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv("./train.csv")
dataset_train, dataset_valid = train_test_split(
    data, test_size=0.1, random_state=42
)

In [6]:
length = []
for row in data['context']:
    # print(row)
    length.append(len(str(row)))

In [7]:
data['length'] = length

In [12]:
data

,id,context,summary,length
0,166996432,가격도 착한데 쌀 품질도 좋아요 비싼 쌀은 6만 원대가 넘어가는데 다섯 식구인 저희...,<맛> 곰곰 우리 쌀이 제일 <가격> 가격만 저렴하고 쌀 품질도 좋음 <향> 구수한...,4585
1,4549886764,왜 이천 쌀 이천 쌀 하는지 알 거 같아요. 일단 쌀알이 작은 편인데 고른 편이에요...,<맛> 쌀알이 작은 편이고 깨진 쌀이 없고 벌레 먹은 쌀도 안 보이는 <식감> 찰기...,4086
2,1392714754,여러 번 씻고 밥을 하니 밥하는 냄새부터 아주 고소합니다. 다 된 밥 열어보니 역시...,<맛> 쌀뜨물 색이 회색빛이 돌지 않고 뽀얀 색 <가격> 더 저렴하고 평은 좋고 <...,2676
3,6839819254,우리 집 친정 친구 집까지 믿을 수 있는 쿠팡 쌀 집니 되었습니다.일반 대형마트나 ...,<맛> 냄새도 안 나고 깨진 거 없고 쌀뜨물도 뽀얀 색 <가격> 가격도 저렴하고 평...,4466
4,6703205199,한 가지만 먹지 않고 이것저것 먹어보는 편이에요. 이번에 체험 기회로 국내 유일 따...,<양> 쌀 3킬로 정도 남은 상태에서 쌀 구입 <크기> 일반 가정에서 먹기에는 큰 ...,2153
...,...,...,...,...
3121,2228201393,파에 단맛도 부족해서 김치 맛도 좀 덜하게 느껴지고요 계절마다 쪽파 실파 맛이 다 ...,<맛> 예소담 특파 김치로 맛있는 <식감> 파 식감이 별로 <양> 가족들이 다 잘 ...,779
3122,1061361421,밥을 너무 많이 먹게 돼요 ㅜㅜㅜ. 남편이 파김치를 좋아해서 가끔 트레이더스에서 사...,<맛> 젓갈 맛이 강한 <양> 대성공이요 <식감> 아삭아삭한 <가격> 저렴하고 <양...,633
3123,6499654111,바로 시식해 봤어요 달지도 짜지도 않더라고요. 젓갈 맛은 느껴지고요 근데 저도 젓갈...,<맛> 달지도 짜지도 않은 <양> 5킬로에 이 가격이면 정말 굿 <파> 싱싱하고 익...,410
3124,1421897625,저는 일단 꽂게 액젓에다가 파대가리를 조금 담가놨다가 양념 버무렸어요 양이 많아서 ...,<맛> 맛있어서 재구매 <양> 3팩 샀더니 양이 엄청 많음 <조리> 액젓에다가 파대...,599


In [14]:
len_filtered_data = data[(data['length'] > 10)]
len_filtered_data

,id,context,summary,length
0,166996432,가격도 착한데 쌀 품질도 좋아요 비싼 쌀은 6만 원대가 넘어가는데 다섯 식구인 저희...,<맛> 곰곰 우리 쌀이 제일 <가격> 가격만 저렴하고 쌀 품질도 좋음 <향> 구수한...,4585
1,4549886764,왜 이천 쌀 이천 쌀 하는지 알 거 같아요. 일단 쌀알이 작은 편인데 고른 편이에요...,<맛> 쌀알이 작은 편이고 깨진 쌀이 없고 벌레 먹은 쌀도 안 보이는 <식감> 찰기...,4086
2,1392714754,여러 번 씻고 밥을 하니 밥하는 냄새부터 아주 고소합니다. 다 된 밥 열어보니 역시...,<맛> 쌀뜨물 색이 회색빛이 돌지 않고 뽀얀 색 <가격> 더 저렴하고 평은 좋고 <...,2676
3,6839819254,우리 집 친정 친구 집까지 믿을 수 있는 쿠팡 쌀 집니 되었습니다.일반 대형마트나 ...,<맛> 냄새도 안 나고 깨진 거 없고 쌀뜨물도 뽀얀 색 <가격> 가격도 저렴하고 평...,4466
4,6703205199,한 가지만 먹지 않고 이것저것 먹어보는 편이에요. 이번에 체험 기회로 국내 유일 따...,<양> 쌀 3킬로 정도 남은 상태에서 쌀 구입 <크기> 일반 가정에서 먹기에는 큰 ...,2153
...,...,...,...,...
3121,2228201393,파에 단맛도 부족해서 김치 맛도 좀 덜하게 느껴지고요 계절마다 쪽파 실파 맛이 다 ...,<맛> 예소담 특파 김치로 맛있는 <식감> 파 식감이 별로 <양> 가족들이 다 잘 ...,779
3122,1061361421,밥을 너무 많이 먹게 돼요 ㅜㅜㅜ. 남편이 파김치를 좋아해서 가끔 트레이더스에서 사...,<맛> 젓갈 맛이 강한 <양> 대성공이요 <식감> 아삭아삭한 <가격> 저렴하고 <양...,633
3123,6499654111,바로 시식해 봤어요 달지도 짜지도 않더라고요. 젓갈 맛은 느껴지고요 근데 저도 젓갈...,<맛> 달지도 짜지도 않은 <양> 5킬로에 이 가격이면 정말 굿 <파> 싱싱하고 익...,410
3124,1421897625,저는 일단 꽂게 액젓에다가 파대가리를 조금 담가놨다가 양념 버무렸어요 양이 많아서 ...,<맛> 맛있어서 재구매 <양> 3팩 샀더니 양이 엄청 많음 <조리> 액젓에다가 파대...,599


In [15]:
len_filtered_data.to_csv('./train_long.csv')